<pre>
> cd /usr/local
> wget python.tgz
> tar -zxvf _tgz
> mv Python-*** Python25
> cd Python25

> #./configure --prefix=/usr/local/Python25
> #make && make install
> #pwd
> /usr/local/Python25
> #python
> >>
</pre>

export PATH=/usr/local/Python25/:$PATH

<pre>
<h1>Hacker's best friend:   ctypes</h1>
<p>
    - ctypes enables you to call dynamically linked libraries
    - extensive capabilities for creating complex C datatypes and utility functions for low-level memory manipulation.
</p>

<h2>Dynamic Libraries</h2>
<p>
    - Compiled Binary that is linked at runtime to the main process executable.
    - WINDOWS   ->  Dynamic Link Libraries  <b>(DLL)</b>
    - Linux     ->  Shared Objects          <b>(SO)</b>
    -   Expose functions through exported names
            which get resolved to actual addresses in memory
    -   Normally at runtime you have to <b><i>resolve the function addresses</b></i> in order to call
            the functions;  however     ctypes      does all the work
</p>

<p>
   3 ways to load Dynamic Libraries in CTYPES:
            
            cdll()
            windll()
            oledll()

    differences in the way those libraries are called and their resulting return values.
</p>

<p>
    cdll()      libraries that export functions using the standard <b><i>cdecl</b></i> calling convention

    windll()    libraries that export functions using the stdcall calling convention
                <b>native of the Microsoft Win32 API</b>
    oledll()    operates exactly like the windll() method <b>BUT</b>
                assumes that the exported functions return a <b><i>Windows HRESULT error</b></i> code
                used specifically for error messages returned from
                <b><i>Microsoft Component Object Model (COM)</b></i>



- C:\WINDOWS\system32\<b>msvcrt.dll</b>    ->      printf()
                    /lib/<b>libc.so.6</b>

In [ ]:
from ctypes import *

msvcrt = cdll.msvcrt

msg = 'Hello World! i am here because of    cdll.msvcrt!\n'

msvcrt.printf("Testing: %s", msg)

https://superuser.com/a/1179434



<img src="img/1_cdll_msvcrtDOTdll.png" style="width:500px;height:180px" />

<img src="img/2_help_cdll.png" style="width:300px;height:200px" />

nix:
-----

from ctypes import *


libc = CDLL('libc.so.6')


msg = 'I am here because of     libc.so.6       usually stored in /lib/'


libc.printf('Testing: %s', msg)

<pre>
<p>
    calling conventions:
        - how to properly call a particular function
            - order of function params
            - which params are      pushed onto the stack
                                            or
                                    passed in registers
            - how the stack is unwound when a function returns
</p>

<p>

2   Calling conventions:

                            cdecl

                            stdcall



    cdecl
            -   params pushed from right to left
            -   caller is responsible for clearing args from the stack
            
            -   most C on x86 


        Example:    (Caller is responsible for clearing args from the stack)

                C
                -
                int python_rocks(reason_one, reason_two, reason_three);

                x86 Assembly
                ---
                push reason_three
                push reason_two
                push reason_one
                call python_rocks
                add esp, 12             ->      Essentially clears those parameters


    stdcall
                -   Order of Params is the same as in cdecl
                -   Function itelf is responsible for clearing out the stack parameters
                
                C
                -
                int my_socks(color_one, color_two, color_three);
                
                x86 Assembly
                ---
                push color_three
                push color_two
                push color_one
                call my_socks           ->      my_socks() responsible for clearing stack

</p>


<p>
    <b>RETURN values are stored in <i>EAX register</i></b>      for both conventions
</p>
</pre>

<pre>


<h3>ctypes</h3>      types can be initialized with a value but
            it has to be of the proper type and size.

</pre>
<img src="img/3_ctypes_c_pythontypes.png" style="width:480px;height:350px" />







<img src="img/4_more_on_ctypes.png" style="width:480px;height:350px" />



The last one is dereferencing the pointer




<pre>
Function that expects a pointer as one of its params


the reason is so that the function can either write to that location in memory
    or
if the parameter is too large, pass by value.





ctypes:     <b>function_main(byref(parameter))</b>



function_main(byref(parameter))
</pre>

<pre>
<h4>Structures and Unions</h4>  -   important Datatypes

<i>structure</i>
                    group of vars
                    same / diff datatypes

        my_struct.member_variable

Defining a structure

C
-
struct beer_recipe
{
    int amt_barley;
    int amt_water;
};

Python
------

class beer_recipe(Structure):
    _fields_ = [
        ("amt_barley", c_int),
        ("amt_water", c_int),
    ]


<i>Union</i>
                    same as structures but,
                    All variables share the <b>same memory location</b>

                    - allows you to specify the same value in different types

C
-
union {
    long    barley_long;
    int     barley_int;
    char    barley_char[8];
}barley_amount;


Python
------
class barley_amount(Union):
    _fields_ = [
        ("barley_long", c_long),
        ("barley_int", c_int),
        ("barley_char", c_char * 8),
    ]


<div style="background-color:black">

>>> from ctypes import *
>>>
>>>
>>>
>>>
>>> class barley_amount(Union):
...     _fields_ = [
...     ('barley_long', c_long),
...     ('barley_int', c_int),
...     ('barley_char', c_char * 8),
...     ]
...
>>>
>>>
>>> value = raw_input("enter the amount of barley to put into the beer vat:")
enter the amount of barley to put into the beer vat:66
>>> my_barley = barley_amount(int(value))
>>>
>>>
>>> print('barley amount as a long: %ld', my_barley.barley_long)
('barley amount as a long: %ld', 66)
>>> print('barley amount as an int: %d', my_barley.barley_long)
('barley amount as an int: %d', 66)
>>> print('barley amount as an int: %d', my_barley.barley_int)
('barley amount as an int: %d', 66)
>>> print('barley amount as a char: %s', my_barley.barley_char)
('barley amount as a char: %s', 'B')


</div>

-> This happens because         B       is      66  in  ASCII




Notice that:    creating an array is ctypes   =    multiply [type] by the number_of_elements
</pre>                    

<pre>
Registers on x86
---------
EAX
EDX
ECX
ESI
EDI
EBP
ESP
EBX



EAX     Accumulator
            - storing RETURN VALUES from function calls
            - calculations    multiplication or division              <-  ONLY WITHIN EAX
                              add, subtract, compare      optimized   <-  possible in others


                -> we can easily figure out if a function worked / failed based on EAX.value
                -> determine actual return value

EDX     Data Register
            - extension of EAX
            - assists in storing extra data for more complex calculations like multi../div
            - can also be used on its own, BUT mostly used WITH EAX

ECX     Count Register              COUNTS <b>DOWNWARDS</b> ONLY
            - Looping operations
                    storing a string
                    counting numbers


            Usually,    ESI     ->      READING     holds the location of Input Stream
                        EDI     ->      WRITING     holds the location where the result is
                                                        stored

ESP     stack   |
EBP     base    |       Managing function calls & stack operations

                        <div style="background-color:pink">
                           When a function is called
                                        
                                        args are pushed onto the stack
                                        followed by RETURN addr


                           ESP points to the very top of the stack
                                        =>      ESP points to the       RETURN ADDRESS

                           EBP points to the bottom of the call stack.
                        </div>
                                in some circumstances a compiler may use optimizations to remove the EBP register as a stack frame pointer;
                                in these cases EBP is freed up to be used like any other general-purpose register.




debugger -> easily read & modify contents of these registers
        
OS API for the debugger to interact with the CPU & retrieve or modify these values















</p>

# #   C:\Users\USER\anaconda3\python.exe:
# print ('python hello')

# To run with python27 change .vscode/settings.json to
# #   "python.pythonPath": "C:\\Python27\\python.exe"

<pre>
Chapter 3:

#   C:\Users\USER\anaconda3\python.exe:
print ('python hello')


#   Our debugger must be able to associate the debugger
#   to the process in some way.

#   -> must be able to either open an executable and run it
#       or attach to a running process.

#   The Win Debugging API provides an easy way to do both


#   There are subtle diff between opening a process
#   & attaching to a process



#   Advantage of opening a process is that you have control
#   of the process before it has a chance to run any code

#   -> handy when analyzing malware


#   Attaching to a process merely breaks into an already running process,
#   which allows you to skip the startup portion of the code and analyze specific areas
#   of code that you are interested in.





 #  Depending on the debugging target and the analysis -> approach



#   The first method of getting a process to run under a debugger is to run the
#   executable from the debugger itself

#   To create a process in Windows you have to call     CreateProcessA()
#       CreateProcessA()    ->  params that enable the process for debugging.




#a call to CreateProcessA()      #   My_LINK:   https://docs.microsoft.com/en-us/windows/win32/api/processthreadsapi/nf-processthreadsapi-createprocessa

# Creates a new process and its primary thread. The new process runs in the security context of the calling process.
# If the calling process is impersonating another user, the new process uses the token for the calling process, not the impersonation token. To run the new process in the security context of the user represented by the impersonation token, use the CreateProcessAsUser or CreateProcessWithLogonW function.


# BOOL WINAPI CreateProcessA(
#     LPCSTR lpApplicationName,
#     LPTSTR lpCommandLine,
#     LPSECURITY_ATTRIBUTES lpProcessAttributes,      #   SECURITY_ATTRIBUTES structure that determines whether the returned handle to the new process object can be inherited by child processes. If lpProcessAttributes is NULL, the handle cannot be inherited.
#     LPSECURITY_ATTRIBUTES lpThreadAttributes,       #   spointer to a SECURITY_ATTRIBUTES structure that determines whether the returned handle to the new thread object can be inherited by child processes. If lpThreadAttributes is NULL, the handle cannot be inherited.
#     BOOL bInheritHandles,
#     DWORD dwCreationFlags,
#     LPVOID lpEnvironment,
#     LPCTSTR lpCurrentDirectory,
#     LPSTARTUPINFO lpStartupInfo,
#     LPPROCESS_INFORMATION lpProcessInformation
# );





#     Looks intimidating but break it down into smaller pieces
#
#
#       -> Params that are important for creating a process under a debugger
#
#           lpApplicationName       |   
#           lpCommandLine           |   set the path of the executable to run and any params
#           dwCreationFlags         special value that indicates that the process should be started as a debugged process
#           lpStartupInfo           |
#           lpProcessInformation    |   pointers to structs STARTUPINFO PROCESS_INFORMATION
#                                                           -> that dictate how the 
#                                                              process should be started as
#                                                              well as provide important
#                                                              info regarding the process
#                                                              after it has been successfully
#                                                              started
#               The rest can be set to NULL
#
#
#
#
#
</pre>